## Data Preprocessing

In [71]:
import pandas as pd
import numpy as np

In [72]:
oral = pd.read_csv('qsar_oral_toxicity.csv', sep=";", header = None)

In [73]:
oral.shape

(8992, 1025)

In [74]:
oral.head()

,0,1,2,3,4,5,6,7,8,9,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,negative
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,negative
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,negative
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,negative
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,negative


In [76]:
(oral.iloc[:, 1024] == "positive").sum()

741

In [77]:
oral = oral.replace("positive", 1)
oral = oral.replace("negative", 0)

In [79]:
(oral.iloc[:, 1024] == 1).sum()

741

In [80]:
oral.head()

,0,1,2,3,4,5,6,7,8,9,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
oral.isnull().sum().sum()

0

No missing value, next we split the training and testing set.

In [93]:
X = oral.iloc[:, :1024].to_numpy()
y = oral.iloc[:, 1024].to_numpy()

In [96]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 111)

### $L_1$ Penalized Logistic Regression

In [175]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state = 0, penalty = "l1", C = 0.5, solver = "saga").fit(X_train, y_train)

D:\Anaconda\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [176]:
clf.score(X_train, y_train)

0.948700125121646

In [177]:
clf.score(X_test, y_test)

0.9316286826014453

In [178]:
np.mean(y_test == 0)

0.9193996664813785

In [179]:
(clf.coef_ != 0).sum()

565

### $L_1$ Penalized Logistic Regression using SGD (batch size $=1$)

In [191]:
# reference: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html
from sklearn.linear_model import SGDClassifier
clf_sgd = SGDClassifier(loss = 'log', penalty = "l1", alpha = 0.001, n_jobs = -1).fit(X_train, y_train)

In [192]:
clf_sgd.score(X_train, y_train)

0.9360489364660086

In [193]:
clf_sgd.score(X_test, y_test)

0.9277376320177877

In [194]:
(clf_sgd.coef_ != 0).sum()

125

For batch size larger than $1$

In [190]:
clf_sgd = SGDClassifier(loss = 'log', penalty = "l1", alpha = 0.001, n_jobs = -1)
clf_sgd.partial_fit(X_train, y_train, classes=np.unique(y_train))

SGDClassifier(alpha=0.001, loss='log', n_jobs=-1, penalty='l1')

## Stochastic Proximal Distance Algorithm

In [195]:
%run source.py

In [197]:
np.eye(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [198]:
np.c_[np.ones(3), np.eye(3)]

array([[1., 1., 0., 0.],
       [1., 0., 1., 0.],
       [1., 0., 0., 1.]])

In [208]:
# to involve an intercept term
Des = np.c_[np.ones(X_train.shape[0]), X_train] # the design matrix
Z = np.c_[y_train, Des] # combine predictors and the response variable to put into the algorithm
n, p = Des.shape

In [209]:
Des.shape

(7193, 1025)

In [210]:
np.zeros(3)

array([0., 0., 0.])

In [375]:
spar = 125
theta0 = np.zeros(p)
rho1 = 1e-2
inc_rate = 1
bsize_init = 0.1 * n

maxiter = 1000
thresh = maxiter
tol = 1e-5
MMsteps = 1


In [376]:
res = SPD_solver(Z, lambda x: Proj_sparsity(x, spar), Prox_logi, loss_logi, theta0, rho1, inc_rate, bsize_init, thresh, MMsteps, tol, maxiter, path_track = False)

In [377]:
res["niter"]

38

In [378]:
np.sum(res["solu"] != 0)

125

In [379]:
Des_test = np.c_[np.ones(X_test.shape[0]), X_test]

tmp = Des_test @ res["solu"]
y_prob = 1 / (1 + np.exp(-tmp))
y_pred = (y_prob > 0.5) * 1

In [380]:
np.mean(y_pred == y_test)

0.9282934963868816